- 환경변수 설정

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_3RAn2d_6bkVakuPSMkobvFRxMZLjKBcmNWLGBdiLCAH9wvaDR6xFNKjvbCh8MgG7RQSU7H'

- Pinecone 클라이언트 초기화

In [18]:
from pinecone import Pinecone

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

In [19]:
# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다.
pc.list_indexes()

[
    {
        "name": "test-embedding-3d",
        "metric": "cosine",
        "host": "test-embedding-3d-dksb1qh.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 3,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "quickstart",
        "metric": "cosine",
        "host": "quickstart-dksb1qh.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1024,
        "deletion_protection": "disabled",
        "tags": null
    },
    {

In [20]:
# 사용할 인덱스 이름을 지정합니다.
index_name = "lol-status"

# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 200}},
 'total_vector_count': 200,
 'vector_type': 'dense'}

검색하기 (Pinecone)

- 질문을 임베딩(벡터로 만들기)

In [21]:
from langchain_openai import OpenAIEmbeddings
 # OpenAIEmbeddings 객체를 초기화합니다.
 # 모델은 "text-embedding-3-small"을 사용하고, API 키는 OPENAI_API_KEY를 사용합니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
# 질문 리스트를 정의합니다.
# question = ["U.S television channel G4"]
question = ["유나라에 대해 알려줘"]
# 질문을 임베딩하여 벡터로 변환합니다.
embedded_question = embedding.embed_documents(question)
# 임베딩된 질문 벡터를 출력합니다.
print(embedded_question)
print(len(embedded_question[0])) # 1536(dimension)

[[-0.014839626848697662, 0.002806924283504486, -0.043025385588407516, 0.03414914757013321, 0.0007871304405853152, -0.009660056792199612, -0.07295884937047958, -0.007504550740122795, -0.014924364164471626, -0.0015093839028850198, -0.0292343832552433, 0.030823208391666412, -0.023790007457137108, -0.017106350511312485, -0.003257091622799635, -0.027793847024440765, -0.027560820803046227, 0.02635331265628338, 0.0590619258582592, 0.01653437316417694, 0.02472211979329586, 0.02569659799337387, -0.03836483135819435, 0.005666809622198343, 0.0040091355331242085, -0.05067292973399162, 0.07867862284183502, 0.02874714322388172, 0.015845881775021553, -0.04268643632531166, -0.026205021888017654, -0.03800469636917114, 0.00878620333969593, 0.0025738966651260853, -0.01163019984960556, 0.0016378138680011034, 0.03584389388561249, -0.06990830600261688, -0.01148191001266241, 0.016756808385252953, 0.002902253996580839, -0.031140971928834915, 0.051901619881391525, 0.016809768974781036, 0.034805864095687866, 0.

- 벡터DB 에서 조회(검색)

In [22]:
# Pinecone 인덱스에서 쿼리를 실행합니다.
query_result = index.query(
    namespace="wiki-ns1",  # wiki-ns1 네임스페이스를 지정합니다.
    vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
    top_k=5,  # 상위 5개의 결과를 반환합니다.
    include_vector=False,  # 벡터를 포함하지 않습니다.
    include_metadata=True  # 메타데이터를 포함합니다.
 )


In [23]:
print(len(query_result.matches))
print(query_result)

5
{'matches': [{'id': 'bbece204-f9cb-4a81-b89b-11d204372631',
              'metadata': {'chunk_id': 0.0,
                           'head_chunk': 'Which resources does Annie use?  '
                                         'Mana',
                           'title': '',
                           'url': '',
                           'wiki_id': '09e9a784-e494-4101-8491-cb45fb9c5629'},
              'score': 0.195750237,
              'values': []},
             {'id': 'a23ac734-e917-4a2b-9400-3a4d1d32cbdd',
              'metadata': {'chunk_id': 0.0,
                           'head_chunk': 'Which resources does Annie use?  '
                                         'Mana',
                           'title': '',
                           'url': '',
                           'wiki_id': '9168ed67-9f38-4a5c-b6e7-91940d9f7f9e'},
              'score': 0.195750237,
              'values': []},
             {'id': '8c1d95a0-5321-418d-a823-3c1dfcd423b7',
              'metadata': {'chunk_

In [24]:
 # 쿼리 결과 query_result 에서 각 매치의 ID를 추출하여 리스트로 만듭니다.
result_ids = [r.id for r in query_result.matches]
print(result_ids)  # 결과 ID를 출력합니다.

['bbece204-f9cb-4a81-b89b-11d204372631', 'a23ac734-e917-4a2b-9400-3a4d1d32cbdd', '8c1d95a0-5321-418d-a823-3c1dfcd423b7', 'fa6d60af-9db1-4056-bfee-85e2814b6afd', 'da3d8a25-b6d4-41b3-9e46-bf062b55a6d8']


In [25]:
# 쿼리 결과에서 각 매치의 점수와 메타데이터를 출력합니다.
for r in query_result.matches:
  print(r.score, r.metadata)

0.195750237 {'chunk_id': 0.0, 'head_chunk': 'Which resources does Annie use?  Mana', 'title': '', 'url': '', 'wiki_id': '09e9a784-e494-4101-8491-cb45fb9c5629'}
0.195750237 {'chunk_id': 0.0, 'head_chunk': 'Which resources does Annie use?  Mana', 'title': '', 'url': '', 'wiki_id': '9168ed67-9f38-4a5c-b6e7-91940d9f7f9e'}
0.184988037 {'chunk_id': 0.0, 'head_chunk': 'What class is Annie?  Mage', 'title': '', 'url': '', 'wiki_id': 'f96c8758-ae9b-471c-8fa5-dd7ce29d3b49'}
0.184988037 {'chunk_id': 0.0, 'head_chunk': 'What class is Annie?  Mage', 'title': '', 'url': '', 'wiki_id': '3da53c99-02ad-44ce-b178-9f85f69db406'}
0.180963516 {'chunk_id': 0.0, 'head_chunk': 'What class is Anivia?  Mage', 'title': '', 'url': '', 'wiki_id': 'f64b4e63-a066-4666-9f4c-6b65341fdc36'}


검색하기 (LangChain)

In [26]:
from langchain_pinecone import PineconeVectorStore
# PineconeVectorStore 객체를 초기화합니다.
# 인덱스는 이전에 정의된 index 객체를 사용하고, 임베딩은 embedding 객체를 사용합니다.
# 텍스트 키는 "head_chunk"로 설정합니다.
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key="head_chunk")
# 질문에 대한 유사도 검색을 수행합니다.
# query는 question 리스트의 첫 번째 요소를 사용하고, 상위 5개의 결과를 반환합니다.
docs = vector_store.similarity_search(query=question[0], k=5, namespace="wiki-ns1")
print(docs)
# 검색된 문서의 메타데이터를 출력합니다.
for doc in docs:
    print(doc.metadata)

[Document(id='bbece204-f9cb-4a81-b89b-11d204372631', metadata={'chunk_id': 0.0, 'title': '', 'url': '', 'wiki_id': '09e9a784-e494-4101-8491-cb45fb9c5629'}, page_content='Which resources does Annie use?  Mana'), Document(id='a23ac734-e917-4a2b-9400-3a4d1d32cbdd', metadata={'chunk_id': 0.0, 'title': '', 'url': '', 'wiki_id': '9168ed67-9f38-4a5c-b6e7-91940d9f7f9e'}, page_content='Which resources does Annie use?  Mana'), Document(id='8c1d95a0-5321-418d-a823-3c1dfcd423b7', metadata={'chunk_id': 0.0, 'title': '', 'url': '', 'wiki_id': 'f96c8758-ae9b-471c-8fa5-dd7ce29d3b49'}, page_content='What class is Annie?  Mage'), Document(id='fa6d60af-9db1-4056-bfee-85e2814b6afd', metadata={'chunk_id': 0.0, 'title': '', 'url': '', 'wiki_id': '3da53c99-02ad-44ce-b178-9f85f69db406'}, page_content='What class is Annie?  Mage'), Document(id='da3d8a25-b6d4-41b3-9e46-bf062b55a6d8', metadata={'chunk_id': 0.0, 'title': '', 'url': '', 'wiki_id': 'f64b4e63-a066-4666-9f4c-6b65341fdc36'}, page_content='What class i

- LLM 프롬프트에서 RAG 결과 활용하기

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model = 'gpt-4.1-mini')

In [28]:
question = "유나라에 대해 알려줘"
# f-string 없이 {context},{question} 은 속성이름으로 쓰입니다.
chat_template = ChatPromptTemplate.from_messages([
    ('system','''당신은 백과사전 처러 여러가지 지식을 갖고 있습니다. 
                질문에 대해 {context} 활용하여 잘 설명해주세요.'''),
    ('user','''{question} 에 대하여 지식을 알려주세요.''')          
])

In [29]:
context = ""
for doc in docs:
  context += doc.page_content
print(context)

Which resources does Annie use?  ManaWhich resources does Annie use?  ManaWhat class is Annie?  MageWhat class is Annie?  MageWhat class is Anivia?  Mage


In [30]:
prompt = chat_template.format_messages(question=question, context=context)
response = llm.invoke(prompt)
print(response.content)

유나라(劉나라)는 중국 역사에서 여러 시기에 존재했던 국가 이름 중 하나로, 주로 한나라(한 왕조)와 관련이 깊습니다. 질문에서 특별한 시대나 문맥이 없으므로, 가장 대표적인 유나라인 ‘한나라’를 중심으로 설명드리겠습니다.

1. 한나라(기원전 206년 ~ 서기 220년)
- 창건자: 유방(유왕)이 진나라 멸망 후 기원전 202년에 세운 왕조입니다.
- 특징: 한나라는 중국 역사상 중요한 왕조 중 하나로, 중앙집권적 통치와 유교를 국교로 삼아 중국의 정치, 경제, 문화 발전에 큰 영향을 끼쳤습니다.
- 통치체제: 황제가 절대 권력을 지니고 태후와 대신들이 보좌하는 체제였으며, 유교 경전을 바탕으로 인재를 등용하는 과거 제도를 시작했습니다.
- 경제와 문화: 농업 생산력이 향상되고, 실크로드를 통해 서역과 활발한 교역이 이루어졌으며, 한자 문화가 정립되어 후대에 지대한 영향을 주었습니다.
- 주요 인물: 유방(한 고조), 한무제(강력한 군주), 장건(서역 개척), 사마천(사기 저술가) 등이 있습니다.

2. 기타 ‘유나라’와 관련된 정보
- 삼국지 시대에도 유씨 가문이 정치적으로 중요한 역할을 했습니다. 예를 들어, 촉한(蜀漢)의 창시자인 유비(유 표시장)이 있습니다.
- 중국 역사상 ‘유나라’로 불리었던 다른 소국이나 부족도 있지만 대표적인 것은 한나라입니다.

만약 특정 시대나 지역의 ‘유나라’를 의미하시면 좀 더 구체적인 정보를 알려주시면 자세히 설명드릴 수 있습니다.
